--- Day 12: Rain Risk ---

Your ferry made decent progress toward the island, but the storm came in faster than anyone expected. The ferry needs to take evasive actions!

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:

    Action N means to move north by the given value.
    Action S means to move south by the given value.
    Action E means to move east by the given value.
    Action W means to move west by the given value.
    Action L means to turn left the given number of degrees.
    Action R means to turn right the given number of degrees.
    Action F means to move forward by the given value in the direction the ship is currently facing.

The ship starts by facing east. Only the L and R actions change the direction the ship is facing. (That is, if the ship is facing east and the next instruction is N10, the ship would move north 10 units, but would still move east if the following action were F.)

For example:

    F10
    N3
    F7
    R90
    F11

These instructions would be handled as follows:

    F10 would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
    N3 would move the ship 3 units north to east 10, north 3.
    F7 would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
    R90 would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
    F11 would move the ship 11 units south to east 17, south 8.

At the end of these instructions, the ship's Manhattan distance (sum of the absolute values of its east/west position and its north/south position) from its starting position is 17 + 8 = 25.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

In [22]:
def read_file_to_arry():
    file1 = open('data/day12.txt', 'r') 
    Lines = file1.readlines()
    data = []
    for line in Lines:
        data.append(line.rstrip('\n'))
    return data

In [11]:
from bidict import bidict
import numpy as np


class Ship:
    
    faces = bidict({'E': 0, 'S':90, 'W':180, 'N':270})
    
    def __init__(self, x, y, face):
        self.x = x
        self.y = y
        self.face = face
        
    def get_manhatten_dist(self):
        return abs(self.x) + abs(self.y)
    
    def get_coords(self):
        return (self.x, self.y)
    
    def turn(self, direction, degrees):
        if (direction == 'L'):
            key = self.faces[self.face] - degrees
            if (key < 0):
                key = 360 + key
            elif (key == 360):
                key = 0
            self.face = self.faces.inverse[key]
        elif (direction == 'R'):
            key = self.faces[self.face] + degrees
            if (key > 360):
                key = key - 360
            elif (key == 360):
                key = 0
            self.face = self.faces.inverse[key]
            
    def move(self, action, value):
        if action == 'F':
            self._move(self.face, value)
        else:
            self._move(action, value)
            
    def _move(self, action, value):
        if action == 'N':
            self.y += 1 * value
        elif action == 'S':
            self.y += -1 * value
        elif action == 'E':
            self.x += -1 * value
        elif action == 'W':
            self.x += 1 * value


In [220]:

def manhatten_distance(ship, data):
    for inst in data:
        action = inst[0]
        value = inst[1:]
        if (action == 'L' or action == 'R'):
            ship.turn(action, int(value))
        else:
            ship.move(action, int(value))

    return ship.get_manhatten_dist()
    
data = read_file_to_arry()
ship = Ship(0,0, 'E')
data = read_file_to_arry()
part_one_answer = manhatten_distance(ship, data)
print("Part one answer: " + str(part_one_answer))

Part one answer: 1032


In [224]:
# testing

ship = Ship(0,0, 'E')
ship.move('F', 10)
ship.move('N', 3)
ship.move('F', 7)
ship.turn('R', 90)
ship.move('F', 11)
print(ship.get_coords())
ship.get_manhatten_dist()

(-17, -8)


25

--- Part Two ---

Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:

    Action N means to move the waypoint north by the given value.
    Action S means to move the waypoint south by the given value.
    Action E means to move the waypoint east by the given value.
    Action W means to move the waypoint west by the given value.
    Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
    Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
    Action F means to move forward to the waypoint a number of times equal to the given value.

The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:

    F10 moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
    N3 moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
    F7 moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
    R90 rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
    F11 moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.

After these operations, the ship's Manhattan distance from its starting position is 214 + 72 = 286.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

In [32]:
from bidict import bidict
import numpy as np

class BetterShip:
    
    def __init__(self, x, y, waypoint):
        self.x = x
        self.y = y
        self.waypoint = waypoint
        
    def get_manhatten_dist(self):
        return abs(self.x) + abs(self.y)
    
    def get_coords(self):
        return (self.x, self.y)
    
    def turn(self, direction, degrees):
        if (direction == 'L'):
            self.waypoint.rotate_left(degrees)
        elif (direction == 'R'):
            self.waypoint.rotate_right(degrees)
            
    def move(self, action, value):
        if action == 'F':
            self.x +=  self.waypoint.x * value
            self.y += self.waypoint.y * value
        else:
            waypoint.udpate_waypoint(action, value)
            
            
class Waypoint:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y 
        
    def get_coords(self):
        return (self.x, self.y)
    
    def udpate_waypoint(self, action, value):
        if action == 'N':
            self.y += value
        elif action == 'S':
            self.y -= value
        elif action == 'E':
            self.x -= value
        elif action == 'W':
            self.x += value
        
    def rotate_right(self, degrees):
        if degrees == 90:
            x = self.y*-1
            y = self.x
            self.x = x
            self.y = y
        elif degrees == 180:
            x = self.x*-1
            y = self.y*-1
            self.x = x
            self.y = y
        elif degrees == 270:
            x = self.y
            y = self.x*-1
            self.x = x
            self.y = y
            
    def rotate_left(self, degrees):
        if degrees == 90:
            x = self.y
            y = self.x*-1
            self.x = x
            self.y = y
        elif degrees == 180:
            x = self.x*-1
            y = self.y*-1
            self.x = x
            self.y = y
        elif degrees == 270:
            x = self.y*-1
            y = self.x
            self.x = x
            self.y = y

In [33]:
# testing

waypoint = Waypoint(-10,1)
ship = BetterShip(0,0, waypoint)
ship.move('F', 10)
print(ship.get_coords())
print(ship.waypoint.get_coords())
ship.move('N', 3)
print(ship.get_coords())
print(ship.waypoint.get_coords())
ship.move('F', 7)
print(ship.get_coords())
print(ship.waypoint.get_coords())
ship.turn('R', 90)
print(ship.get_coords())
print(ship.waypoint.get_coords())
ship.move('F', 11)
print(ship.get_coords())
ship.get_manhatten_dist()

(-100, 10)
(-10, 1)
(-100, 10)
(-10, 4)
(-170, 38)
(-10, 4)
(-170, 38)
(-4, -10)
(-214, -72)


286

In [34]:
def manhatten_distance_2(ship, data):
    for inst in data:
        action = inst[0]
        value = inst[1:]
        if (action == 'L' or action == 'R'):
            ship.turn(action, int(value))
        else:
            ship.move(action, int(value))

    return ship.get_manhatten_dist()
    
waypoint = Waypoint(-10,1)
ship = BetterShip(0,0, waypoint)
data = read_file_to_arry()
part_two_answer = manhatten_distance_2(ship, data)
print("Part two answer: " + str(part_two_answer))

Part two answer: 156735
